### Importing Libraries

In [81]:
import pandas as pd
import numpy as np
import re

### Loading Data

In [82]:
df = pd.read_csv("https://raw.githubusercontent.com/Basharat-Asghar/PakWheels-car-Price-Prediction/refs/heads/main/data/pakwheels_cars_raw.csv")

In [83]:
df.head()

,Title,Price,Year,Mileage,Fuel,Engine,Transmission
0,Porsche Cayenne 2016 S E-Hybrid Platinum Edition,PKR 2.35crore,2016,"75,943 km",Hybrid,3000 cc,Automatic
1,Proton X70 2022 Premium FWD,PKR 70lacs,2022,"12,319 km",Petrol,1500 cc,Automatic
2,FAW X-PV 2014 Dual AC,PKR 7lacs,2014,"331,849 km",Petrol,1000 cc,Manual
3,Honda N Wgn 2023 G Turbo,PKR 40lacs,2023,"25,762 km",Petrol,660 cc,Automatic
4,Suzuki Wagon R 2017 VXR,PKR 17.5lacs,2017,"147,219 km",Petrol,1000 cc,Manual


### Data Cleaning

In [84]:
df.columns

Index(['Title', 'Price', 'Year', 'Mileage', 'Fuel', 'Engine', 'Transmission'], dtype='object')

Converting column names to lower case.

In [85]:
df.columns = df.columns.str.lower()

In [86]:
df.columns

Index(['title', 'price', 'year', 'mileage', 'fuel', 'engine', 'transmission'], dtype='object')

Extracting price from string.

In [87]:
def extract_convert_price(value):
  value = value.lower().replace("pkr", "").strip()
  if "crore" in value:
    num = float(value.replace("crore", "").strip())
    return num * 100
  elif "lac" in value or "lacs" in value:
    num = float(value.replace("lacs", "").replace("lac", "").strip())
    return num

  return None

In [115]:
df["price_pkr_lacs"] = pd.to_numeric(df['price'].apply(extract_convert_price))
df.head()

,title,price,year,mileage,fuel,engine,transmission,price_pkr_lacs,mileage_km,engine_numeric,engine_type,company,model,drive_type
0,Porsche Cayenne 2016 S E-Hybrid Platinum Edition,PKR 2.35crore,2016,"75,943 km",Hybrid,3000 cc,Automatic,235.0,75943,3000.0,ICE,Porsche,Cayenne,NaN
1,Proton X70 2022 Premium FWD,PKR 70lacs,2022,"12,319 km",Petrol,1500 cc,Automatic,70.0,12319,1500.0,ICE,Proton,X70,FWD
2,FAW X-PV 2014 Dual AC,PKR 7lacs,2014,"331,849 km",Petrol,1000 cc,Manual,7.0,331849,1000.0,ICE,Faw,X-PV,NaN
3,Honda N Wgn 2023 G Turbo,PKR 40lacs,2023,"25,762 km",Petrol,660 cc,Automatic,40.0,25762,660.0,ICE,Honda,N Wgn,NaN
4,Suzuki Wagon R 2017 VXR,PKR 17.5lacs,2017,"147,219 km",Petrol,1000 cc,Manual,17.5,147219,1000.0,ICE,Suzuki,Wagon R,NaN


Extracting kms_driven from mileage column.

In [89]:
def extract_mileage(value):
  value = value.lower().replace("km", "").replace(",", "").strip()
  return value

In [90]:
df["mileage_km"] = pd.to_numeric(df['mileage'].apply(extract_mileage))
df.head()

,title,price,year,mileage,fuel,engine,transmission,price_pkr_lacs,mileage_km
0,Porsche Cayenne 2016 S E-Hybrid Platinum Edition,PKR 2.35crore,2016,"75,943 km",Hybrid,3000 cc,Automatic,235.0,75943
1,Proton X70 2022 Premium FWD,PKR 70lacs,2022,"12,319 km",Petrol,1500 cc,Automatic,70.0,12319
2,FAW X-PV 2014 Dual AC,PKR 7lacs,2014,"331,849 km",Petrol,1000 cc,Manual,7.0,331849
3,Honda N Wgn 2023 G Turbo,PKR 40lacs,2023,"25,762 km",Petrol,660 cc,Automatic,40.0,25762
4,Suzuki Wagon R 2017 VXR,PKR 17.5lacs,2017,"147,219 km",Petrol,1000 cc,Manual,17.5,147219


Cleaning engine column to extract numeric value of engine and engine type.

In [91]:
def extract_engine(row):
  fuel = str(row['fuel']).lower().strip()
  engine_val = str(row['engine']).lower().strip()
  if fuel == 'electric':
    num = re.findall(r"\d+\.?\d*", engine_val)
    return (float(num[0]) if num else None, "EV")
  else:
    num = re.findall(r"\d+\.?\d*", engine_val)
    return (float(num[0]) if num else None, "ICE")

In [92]:
df[["engine_numeric", "engine_type"]] = df.apply(extract_engine, axis=1, result_type="expand")
df.head()

,title,price,year,mileage,fuel,engine,transmission,price_pkr_lacs,mileage_km,engine_numeric,engine_type
0,Porsche Cayenne 2016 S E-Hybrid Platinum Edition,PKR 2.35crore,2016,"75,943 km",Hybrid,3000 cc,Automatic,235.0,75943,3000.0,ICE
1,Proton X70 2022 Premium FWD,PKR 70lacs,2022,"12,319 km",Petrol,1500 cc,Automatic,70.0,12319,1500.0,ICE
2,FAW X-PV 2014 Dual AC,PKR 7lacs,2014,"331,849 km",Petrol,1000 cc,Manual,7.0,331849,1000.0,ICE
3,Honda N Wgn 2023 G Turbo,PKR 40lacs,2023,"25,762 km",Petrol,660 cc,Automatic,40.0,25762,660.0,ICE
4,Suzuki Wagon R 2017 VXR,PKR 17.5lacs,2017,"147,219 km",Petrol,1000 cc,Manual,17.5,147219,1000.0,ICE


Extract company name from title column.

In [93]:
car_companies = [
    "Mercedes Benz", "Land Rover", "Range Rover",
    "Porsche", "Proton", "FAW", "Honda", "Suzuki", "Toyota", "Changan",
    "KIA", "Nissan", "Lexus", "Daihatsu", "Isuzu", "Mitsubishi",
    "BMW", "Audi", "Hyundai", "Jeep", "Subaru", "Ford", "Chevrolet"
]

In [94]:
pattern = r"\b(" + "|".join(sorted(car_companies, key=lambda x: -len(x))) + r")\b"
df["company"] = df["title"].str.extract(pattern, flags=re.IGNORECASE)
df["company"] = df["company"].fillna(df["title"].str.split().str[0])
df["company"] = df["company"].str.title()
df.head()

,title,price,year,mileage,fuel,engine,transmission,price_pkr_lacs,mileage_km,engine_numeric,engine_type,company
0,Porsche Cayenne 2016 S E-Hybrid Platinum Edition,PKR 2.35crore,2016,"75,943 km",Hybrid,3000 cc,Automatic,235.0,75943,3000.0,ICE,Porsche
1,Proton X70 2022 Premium FWD,PKR 70lacs,2022,"12,319 km",Petrol,1500 cc,Automatic,70.0,12319,1500.0,ICE,Proton
2,FAW X-PV 2014 Dual AC,PKR 7lacs,2014,"331,849 km",Petrol,1000 cc,Manual,7.0,331849,1000.0,ICE,Faw
3,Honda N Wgn 2023 G Turbo,PKR 40lacs,2023,"25,762 km",Petrol,660 cc,Automatic,40.0,25762,660.0,ICE,Honda
4,Suzuki Wagon R 2017 VXR,PKR 17.5lacs,2017,"147,219 km",Petrol,1000 cc,Manual,17.5,147219,1000.0,ICE,Suzuki


Extracting model from title.

In [95]:
stop_patterns = [
    r"^\d{4}$",        # year
    r"hybrid",         # fuel/variant
    r"automatic", r"manual", r"cv[tp]?", r"ags", r"prosmatec", r"tiptronic",
    r"auto gear shift", r"cruisetronic", r"awd", r"fwd", r"rwd", r"4wd"
]
stop_regex = re.compile("|".join(stop_patterns), re.IGNORECASE)

In [96]:
def extract_model(title, company):
  if not isinstance(title, str) or not company:
      return None

  remaining = re.sub(company, "", title, flags=re.IGNORECASE).strip()
  words = re.findall(r"[A-Za-z0-9\-]+", remaining)

  model_words = []
  for w in words:
    if stop_regex.match(w):
      break
    model_words.append(w)

  return " ".join(model_words) if model_words else None

In [97]:
df["model"] = df.apply(lambda x: extract_model(x["title"], x["company"]), axis=1)
df.head()

,title,price,year,mileage,fuel,engine,transmission,price_pkr_lacs,mileage_km,engine_numeric,engine_type,company,model
0,Porsche Cayenne 2016 S E-Hybrid Platinum Edition,PKR 2.35crore,2016,"75,943 km",Hybrid,3000 cc,Automatic,235.0,75943,3000.0,ICE,Porsche,Cayenne
1,Proton X70 2022 Premium FWD,PKR 70lacs,2022,"12,319 km",Petrol,1500 cc,Automatic,70.0,12319,1500.0,ICE,Proton,X70
2,FAW X-PV 2014 Dual AC,PKR 7lacs,2014,"331,849 km",Petrol,1000 cc,Manual,7.0,331849,1000.0,ICE,Faw,X-PV
3,Honda N Wgn 2023 G Turbo,PKR 40lacs,2023,"25,762 km",Petrol,660 cc,Automatic,40.0,25762,660.0,ICE,Honda,N Wgn
4,Suzuki Wagon R 2017 VXR,PKR 17.5lacs,2017,"147,219 km",Petrol,1000 cc,Manual,17.5,147219,1000.0,ICE,Suzuki,Wagon R


Drive type extraction.

In [98]:
df["drive_type"] = df["title"].str.extract(r"\b(AWD|4WD|FWD|RWD)\b", flags=re.IGNORECASE)
df.head()

,title,price,year,mileage,fuel,engine,transmission,price_pkr_lacs,mileage_km,engine_numeric,engine_type,company,model,drive_type
0,Porsche Cayenne 2016 S E-Hybrid Platinum Edition,PKR 2.35crore,2016,"75,943 km",Hybrid,3000 cc,Automatic,235.0,75943,3000.0,ICE,Porsche,Cayenne,NaN
1,Proton X70 2022 Premium FWD,PKR 70lacs,2022,"12,319 km",Petrol,1500 cc,Automatic,70.0,12319,1500.0,ICE,Proton,X70,FWD
2,FAW X-PV 2014 Dual AC,PKR 7lacs,2014,"331,849 km",Petrol,1000 cc,Manual,7.0,331849,1000.0,ICE,Faw,X-PV,NaN
3,Honda N Wgn 2023 G Turbo,PKR 40lacs,2023,"25,762 km",Petrol,660 cc,Automatic,40.0,25762,660.0,ICE,Honda,N Wgn,NaN
4,Suzuki Wagon R 2017 VXR,PKR 17.5lacs,2017,"147,219 km",Petrol,1000 cc,Manual,17.5,147219,1000.0,ICE,Suzuki,Wagon R,NaN


Removing extreme price values based on model, 4 * median price.

In [111]:
price_median_per_model = df.groupby('model')['price_pkr_lacs'].transform('median')
df['price_pkr_lacs'] = df['price_pkr_lacs'].where(df['price_pkr_lacs'] <= 4 * price_median_per_model, price_median_per_model)

In [131]:
df1 = df.copy()

Cleaning engine_numeric column to get rid of unrealistic values with the median of per model type.

In [137]:
ice_mask = df1['engine_type'] == 'ICE'
unrealistic_ice = (df1['engine_numeric'] < 600) | \
                  (df1['engine_numeric'] > 8000) | \
                  (df1['engine_numeric'] == 0)
df1.loc[ice_mask & unrealistic_ice, 'engine_numeric'] = np.nan

In [148]:
ev_mask = df1['engine_type'].isin(['EV'])
unrealistic_ev = (df1['engine_numeric'] < 20) | \
                  (df1['engine_numeric'] > 200) | \
                  (df1['engine_numeric'] == 0)
df1.loc[ev_mask & unrealistic_ev, 'engine_numeric'] = np.nan

In [149]:
median_per_model_type = df1.groupby(['model', 'engine_type'])['engine_numeric'].transform('median')
df1['engine_numeric'] = df1['engine_numeric'].fillna(median_per_model_type)

Dropping columns.

In [152]:
df1.drop(columns=['price', 'mileage', 'engine'], inplace=True)
df1.head()

,title,year,fuel,transmission,price_pkr_lacs,mileage_km,engine_numeric,engine_type,company,model,drive_type
0,Porsche Cayenne 2016 S E-Hybrid Platinum Edition,2016,Hybrid,Automatic,235.0,75943,3000.0,ICE,Porsche,Cayenne,NaN
1,Proton X70 2022 Premium FWD,2022,Petrol,Automatic,70.0,12319,1500.0,ICE,Proton,X70,FWD
2,FAW X-PV 2014 Dual AC,2014,Petrol,Manual,7.0,331849,1000.0,ICE,Faw,X-PV,NaN
3,Honda N Wgn 2023 G Turbo,2023,Petrol,Automatic,40.0,25762,660.0,ICE,Honda,N Wgn,NaN
4,Suzuki Wagon R 2017 VXR,2017,Petrol,Manual,17.5,147219,1000.0,ICE,Suzuki,Wagon R,NaN


In [153]:
df1.to_csv("pakwheels_cleaned_data.csv", index=False)